## Intro to MAUP

@author: eveomett AI for Redistricting, USF All data retrieved 04/30/24:
https://redistrictingdatahub.org/dataset/virginia-block-pl-94171-2020-by-table/
https://redistrictingdatahub.org/dataset/vest-2020-virginia-precinct-boundaries-and-election-results-shapefile/
https://redistrictingdatahub.org/dataset/2021-virginia-congressional-districts-approved-plan/

https://redistrictingdatahub.org/dataset/vest-2018-virginia-precinct-and-election-results/
https://redistrictingdatahub.org/dataset/vest-2016-virginia-precinct-and-election-results/
https://redistrictingdatahub.org/dataset/vest-2021-virginia-precinct-boundaries-and-election-results-shapefile/
https://redistrictingdatahub.org/dataset/vest-2017-virginia-precinct-boundaries-and-election-results-shapefile/

https://redistrictingdatahub.org/dataset/2021-senate-of-virginia-districts-approved-plan/
https://redistrictingdatahub.org/dataset/2021-virginia-house-of-delegates-districts-approved-plan/

In [1]:
import pandas as pd
import geopandas as gpd
import maup
import time
from maup import smart_repair
from gerrychain import Graph

maup.progress.enabled = True

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# state = Virginia
state_ab = "va"

## Data
1. Download all the data in directory "il_data"
2. Extract them all

In [4]:
data_folder = state_ab + "_data/"
population1_data = "./{}{}_pl2020_b-2/{}_pl2020_p1_b.shp".format(data_folder, state_ab, state_ab)
population2_data = "./{}{}_pl2020_b-2/{}_pl2020_p2_b.shp".format(data_folder, state_ab, state_ab)
vap_data =  "./{}{}_pl2020_b-2/{}_pl2020_p4_b.shp".format(data_folder, state_ab, state_ab)
vest21_data = "./{}{}_vest_21/{}_vest_21.shp".format(data_folder, state_ab, state_ab)
vest20_data = "./{}{}_vest_20/{}_vest_20.shp".format(data_folder, state_ab, state_ab)
vest18_data = "./{}{}_vest_18/{}_vest_18.shp".format(data_folder, state_ab, state_ab)
vest17_data = "./{}{}_vest_17/{}_vest_17.shp".format(data_folder, state_ab, state_ab)
vest16_data = "./{}{}_vest_16/{}_vest_16.shp".format(data_folder, state_ab, state_ab)
cd_data = "./{}{}_cong_adopted_2021/SCV FINAL CD.shp".format(data_folder, state_ab)
send_data = "./{}{}_sldu_adopted_2021/SCV FINAL SD.shp".format(data_folder, state_ab)
hdist_data = "./{}{}_sldl_adopted_2021/SCV FINAL HOD.shp".format(data_folder, state_ab)

In [5]:
def do_smart_repair(df):
    # change it to the UTM it needs for smart_repair
    df = df.to_crs(df.estimate_utm_crs())
    df = smart_repair(df)
    if maup.doctor(df):
        print('smart_repair successful')
            
        # change it back to this UTM for this data
        df = df.to_crs('EPSG:4269')
    else:
        print('smart_repair failed')
    return df

In [6]:
def do_nested_smart_repair(df, cong_df):
    # change it to the UTM it needs for smart_repair
    df = df.to_crs(df.estimate_utm_crs())
    cong_df = cong_df.to_crs(df.estimate_utm_crs())
    
    df = smart_repair(df, nest_within_regions = cong_df)
    if maup.doctor(df):
        print('smart_repair successful')
            
        # change it back to this UTM for this data
        cong_df = cong_df.to_crs('EPSG:4269')
        cong_df = cong_df.to_crs('EPSG:4269')
    else:
        print('smart_repair failed')
    return df

In [7]:
def add_district(dist_df, dist_name, election_df, col_name):
    # check if it needs to be smart_repair
    if maup.doctor(dist_df) != True:
        dist_df = do_smart_repair(dist_df)
    
    election_df = gpd.GeoDataFrame(election_df, crs="EPSG:4269")
    
    # assigne the pricincts
    precincts_to_district_assignment = maup.assign(election_df.geometry, dist_df.geometry)
    election_df[dist_name] = precincts_to_district_assignment
    for precinct_index in range(len(election_df)):
        election_df.at[precinct_index, dist_name] = dist_df.at[election_df.at[precinct_index, dist_name], col_name]
    
    return election_df

In [8]:
def rename(original, year):
    party = original[6]
    if party == 'R' or party == 'D':
        return original[3:6] + year + original[6]
    else:
        return original[3:6] + year + 'O'

In [23]:
def rename_vest_data(vest_data, year):
    vest = gpd.read_file(vest_data)
    
     # check if it needs to be smart_repair
    if maup.doctor(vest) != True:
        vest = do_smart_repair(vest)
    
    # rename the columns
    original_col = vest.columns[5:-1]
    new_col = [rename(i, year) for i in original_col]
    rename_dict = dict(zip(original_col, new_col))
    vest = vest.rename(columns=rename_dict)
    vest = vest.groupby(level=0, axis=1).sum() # combine all the other party's vote into columns with sufix "O"
    col_name = list(set(new_col))
    col_name.sort()
    
    return vest

In [10]:
def blocks_to_precincts(block, df):
    # assign block to vest
    block = gpd.GeoDataFrame(block, crs="EPSG:4269")
    df = gpd.GeoDataFrame(df, crs="EPSG:4269")
    block_to_precinct_assginment = maup.assign(block.geometry, df.geometry)
    return block_to_precinct_assginment

### Read the census data

In [11]:
population1_df = gpd.read_file(population1_data)
population2_df = gpd.read_file(population2_data)
vap_df = gpd.read_file(vap_data)

In [12]:
population2_df = population2_df.drop(columns=['SUMLEV', 'LOGRECNO', 'GEOID', 'COUNTY', 'geometry'])
vap_df = vap_df.drop(columns=['SUMLEV', 'LOGRECNO', 'GEOID', 'COUNTY', 'geometry'])

In [13]:
population_df = pd.merge(population1_df, population2_df, on='GEOID20')
population_df = pd.merge(population_df, vap_df, on='GEOID20')

In [14]:
maup.doctor(population_df)

100%|██████████| 163491/163491 [02:50<00:00, 956.18it/s] 


True

In [15]:
population_df['H_WHITE'] = population_df.apply(lambda t: t['P0010003'] - t['P0020005'], 1)
population_df['H_BLACK'] = population_df.apply(lambda t: t['P0010004'] - t['P0020006'], 1)
population_df['H_AMIN'] = population_df.apply(lambda t: t['P0010005'] - t['P0020007'], 1)
population_df['H_ASIAN'] = population_df.apply(lambda t: t['P0010006'] - t['P0020008'], 1)
population_df['H_NHPI'] = population_df.apply(lambda t: t['P0010007'] - t['P0020009'], 1)
population_df['H_OTHER'] = population_df.apply(lambda t: t['P0010008'] - t['P0020010'], 1)
population_df['H_2MORE'] = population_df.apply(lambda t: t['P0010009'] - t['P0020011'], 1)

In [20]:
cong_df = gpd.read_file(cd_data)
maup.doctor(cong_df)

100%|██████████| 11/11 [00:00<00:00, 17.76it/s]


True

## Read the vest 20 data

Now using it as a "base pricinct", but it could be vest 18 or vest 16 if vest 20 is not working

In [46]:
vest20 = rename_vest_data(vest20_data, '20')
vest20

100%|██████████| 2477/2477 [00:08<00:00, 275.38it/s]


There are 58 overlaps.
There are 587 holes.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|██████████| 2623/2623 [00:02<00:00, 1169.13it/s]


Resolving overlaps...
Assigning order 2 pieces...
Assigning order 3 pieces...
Filling gaps...


100%|██████████| 2477/2477 [00:07<00:00, 334.26it/s]


smart_repair successful


100%|██████████| 2477/2477 [00:09<00:00, 268.61it/s]


True


In [27]:
vest21 = rename_vest_data(vest21_data, '21')
vest18 = rename_vest_data(vest18_data, '18')
vest17 = rename_vest_data(vest17_data, '17')
vest16 = rename_vest_data(vest16_data, '16')

100%|██████████| 2479/2479 [00:11<00:00, 214.51it/s]


There are 58 overlaps.
There are 588 holes.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|██████████| 2624/2624 [00:01<00:00, 1414.02it/s]


Resolving overlaps...
Assigning order 2 pieces...
Assigning order 3 pieces...
Filling gaps...


100%|██████████| 2479/2479 [00:08<00:00, 299.72it/s]


...

There are 36 overlaps.
There are 89 holes.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|██████████| 2571/2571 [00:02<00:00, 1166.10it/s]


Resolving overlaps...
Assigning order 2 pieces...
Assigning order 3 pieces...
Filling gaps...


100%|██████████| 2456/2456 [00:08<00:00, 291.36it/s]


smart_repair successful


### In order to move all this data around, we'll need assignments of blocks to precincts for each of the precinct files.

In [30]:
blocks_to_precincts2021_assignment = blocks_to_precincts(population_df.geometry, vest21.geometry)
blocks_to_precincts2020_assignment = blocks_to_precincts(population_df.geometry, vest20.geometry)
blocks_to_precincts2018_assignment = blocks_to_precincts(population_df.geometry, vest18.geometry)
blocks_to_precincts2017_assignment = blocks_to_precincts(population_df.geometry, vest17.geometry)
blocks_to_precincts2016_assignment = blocks_to_precincts(population_df.geometry, vest16.geometry)

100%|██████████| 2456/2456 [00:39<00:00, 62.38it/s] 


### First step: Aggregate population data from blocks to 2020 precincts.
### (We'll just use a few of the population columns for this demo.)

In [31]:
for column in population_df.columns:
    print(column)

GEOID20
SUMLEV
LOGRECNO
GEOID
COUNTY
P0010001
P0010002
P0010003
P0010004
P0010005
P0010006
P0010007
P0010008
P0010009
P0010010
P0010011
P0010012
P0010013
P0010014
P0010015
P0010016
P0010017
P0010018
P0010019
P0010020
P0010021
P0010022
P0010023
P0010024
P0010025
P0010026
P0010027
P0010028
P0010029
P0010030
P0010031
P0010032
P0010033
P0010034
P0010035
P0010036
P0010037
P0010038
P0010039
P0010040
P0010041
P0010042
P0010043
P0010044
P0010045
P0010046
P0010047
P0010048
P0010049
P0010050
P0010051
P0010052
P0010053
P0010054
P0010055
P0010056
P0010057
P0010058
P0010059
P0010060
P0010061
P0010062
P0010063
P0010064
P0010065
P0010066
P0010067
P0010068
P0010069
P0010070
P0010071
geometry
P0020001
P0020002
P0020003
P0020004
P0020005
P0020006
P0020007
P0020008
P0020009
P0020010
P0020011
P0020012
P0020013
P0020014
P0020015
P0020016
P0020017
P0020018
P0020019
P0020020
P0020021
P0020022
P0020023
P0020024
P0020025
P0020026
P0020027
P0020028
P0020029
P0020030
P0020031
P0020032
P0020033
P0020034
P0020035


In [32]:
vap_df.columns

Index(['GEOID20', 'P0040001', 'P0040002', 'P0040003', 'P0040004', 'P0040005',
       'P0040006', 'P0040007', 'P0040008', 'P0040009', 'P0040010', 'P0040011',
       'P0040012', 'P0040013', 'P0040014', 'P0040015', 'P0040016', 'P0040017',
       'P0040018', 'P0040019', 'P0040020', 'P0040021', 'P0040022', 'P0040023',
       'P0040024', 'P0040025', 'P0040026', 'P0040027', 'P0040028', 'P0040029',
       'P0040030', 'P0040031', 'P0040032', 'P0040033', 'P0040034', 'P0040035',
       'P0040036', 'P0040037', 'P0040038', 'P0040039', 'P0040040', 'P0040041',
       'P0040042', 'P0040043', 'P0040044', 'P0040045', 'P0040046', 'P0040047',
       'P0040048', 'P0040049', 'P0040050', 'P0040051', 'P0040052', 'P0040053',
       'P0040054', 'P0040055', 'P0040056', 'P0040057', 'P0040058', 'P0040059',
       'P0040060', 'P0040061', 'P0040062', 'P0040063', 'P0040064', 'P0040065',
       'P0040066', 'P0040067', 'P0040068', 'P0040069', 'P0040070', 'P0040071',
       'P0040072', 'P0040073'],
      dtype='object')

In [33]:
pop_cols = ['P0020001', 'P0040001']

### We can use the assignment of blocks to precincts to aggregate populations from blocks up to precincts:

In [34]:
vest21[pop_cols] = population_df[pop_cols].groupby(blocks_to_precincts2021_assignment).sum()

In [35]:
vest21[pop_cols].head()

,P0020001,P0040001
0,3345.0,2917.0
1,1511.0,1222.0
2,2887.0,2441.0
3,3560.0,2786.0
4,1378.0,1090.0


#### Check that we didn't gain/lose any population in the aggregation step:


In [36]:
population_df[pop_cols].sum()

P0020001    8631393
P0040001    6745054
dtype: int64

In [37]:
vest21[pop_cols].sum()

P0020001    8631393.0
P0040001    6745054.0
dtype: float64

#### Since we see above the Populations do not match, so lets try with vest 20 dataset

In [38]:
vest20[pop_cols] = population_df[pop_cols].groupby(blocks_to_precincts2020_assignment).sum()

In [39]:
print(population_df[pop_cols].sum())
print(vest20[pop_cols].sum())

P0020001    8631393
P0040001    6745054
dtype: int64
P0020001    8631393.0
P0040001    6745054.0
dtype: float64


In [40]:
vest18[pop_cols] = population_df[pop_cols].groupby(blocks_to_precincts2018_assignment).sum()
print(population_df[pop_cols].sum())
print(vest18[pop_cols].sum())

P0020001    8631393
P0040001    6745054
dtype: int64
P0020001    8631393.0
P0040001    6745054.0
dtype: float64


In [41]:
vest17[pop_cols] = population_df[pop_cols].groupby(blocks_to_precincts2017_assignment).sum()
print(population_df[pop_cols].sum())
print(vest17[pop_cols].sum())

P0020001    8631393
P0040001    6745054
dtype: int64
P0020001    8631393.0
P0040001    6745054.0
dtype: float64


In [42]:
vest16[pop_cols] = population_df[pop_cols].groupby(blocks_to_precincts2016_assignment).sum()
print(population_df[pop_cols].sum())
print(vest16[pop_cols].sum())

P0020001    8631393
P0040001    6745054
dtype: int64
P0020001    8631393
P0040001    6745054
dtype: int64
